In [ ]:
from reggae.models import TranscriptionLikelihood, Options
from reggae.data_loaders import load_barenco_puma, load_3day_dros, DataHolder, scaled_barenco_data
from reggae.utilities import get_rbf_dist, discretise
from scipy.interpolate import interp1d
from sklearn import preprocessing

import tensorflow as tf
from tensorflow import math as tfm
import numpy as np
from matplotlib import pyplot as plt

np.set_printoptions(formatter={'float': lambda x: "{0:0.5f}".format(x)})
plt.style.use('ggplot')
%matplotlib inline
f64 = np.float64


In [ ]:
m_observed, f_observed, σ2_m_pre, σ2_f_pre, t = load_barenco_puma()

# m_observed, f_observed, t = load_3day_dros()

replicate = 0

m_df, m_observed = m_observed 
f_df, f_observed = f_observed

# Shape of m_observed = (replicates, genes, times)
m_observed = m_observed[replicate]
f_observed = np.atleast_2d(f_observed[replicate])
σ2_m_pre = σ2_m_pre[0]
σ2_f_pre = σ2_f_pre[0]

num_genes = m_observed.shape[0]
τ, common_indices = discretise(t)
N_p = τ.shape[0]
N_m = m_observed.shape[1]

f_observed = tf.stack([f_observed[0], f_observed[0], f_observed[0]])

data = (m_observed, f_observed)
noise_data = (σ2_m_pre, σ2_f_pre)
time = (t, τ, (common_indices))

data = DataHolder(data, noise_data, time)
N_p = τ.shape[0]

opt = Options(preprocessing_variance=True, tf_mrna_present=True)
lik = TranscriptionLikelihood(data, opt)
num_tfs = 3

In [ ]:
# Transcription factor
x = data.t
A = np.array([0.01, 0.3, 0.55, 0.4, 0.45, 0.34, 0.015])
B = np.array([0.01, 0.14, 0.3, 0.43, 0.37, 0.2, 0.013])
C = np.array([0.01, 0.01, 0.05, 0.21, 0.41, 0.23, 0.02])
interp = interp1d(np.arange(A.shape[0]), A, kind='cubic')
A = interp(np.linspace(0,6, τ.shape[0]))
interp = interp1d(np.arange(B.shape[0]), B, kind='cubic')
B = interp(np.linspace(0,6, τ.shape[0]))
interp = interp1d(np.arange(C.shape[0]), C, kind='cubic')
C = interp(np.linspace(0,6, τ.shape[0]))

δbar = f64(np.array([-2, -2, -2]))
fbar = np.array([A, B, C])
fbar = preprocessing.normalize(fbar)
print(fbar.shape)
tf_labels = ['A', 'B', 'C']
plt.title('TFs')
for i in range(3):
    plt.plot(np.arange(τ.shape[0]), fbar[i], label=tf_labels[i])
plt.legend()

fbar = tfm.log((tfm.exp(fbar)-1))

plt.figure()
plt.title('TF Proteins')

f_i = tfm.log(1+tfm.exp(fbar))
f = tfm.log(1+tfm.exp(fbar[0]))

p = lik.calculate_protein(fbar, δbar)
plt.plot(p[0], label='A')
plt.plot(p[1], label='B')
plt.plot(p[2], label='C')
plt.legend()

In [ ]:
# Kinetic parameters
#   [[initial, base, sens, decay]]
kbar = np.log(np.array([[0.35, 0.5, 1.5, 0.5],
                        [0.3, 0.4, 2.0, 1.0],
                        [0.34, 0.6, 2.5, 1.2],
                        [0.36, 0.45, 2.0, 0.6],
                        [0.38, 0.7, 1.3, 1.7],]))

kbar = np.log(np.array([[0.36674, 0.47425, 0.74471, 1.55667],
                        [0.30336, 0.44227, 0.78838, 0.82612],
                        [0.22161, 0.34086, 0.99499, 1.89794],
                        [0.34286, 0.26860, 1.47101, 0.92054],
                        [0.26301, 0.35278, 1.04252, 2.37568]]))

In [ ]:
w = 1*tf.ones((num_genes, num_tfs), dtype='float64') # TODO
w_0 = tf.zeros(num_genes, dtype='float64') # TODO

m_pred = lik.predict_m(kbar, δbar, w, fbar, w_0)

plt.figure(figsize=((10, 5)))
print(m_pred.shape)
for j in range(num_genes):
    plt.plot(m_pred[j], label=j)
    
plt.legend()